### Year 3 Topology 3 - Limit Checking - Overloading, Voltage limits

In [1]:
import pandas as pd
import altair as alt

##### Loading

##### Percentage Loading in Branches 

Calculation related can be found in PSSE PAGVI Manual Chapter 6 Power system network simulations - Section 6.6.7 Activity Rate 

%I = MVA(actual)/(MVA(rated)*Vpu)

Vpu -> Bus Voltage at from end 

In [2]:
list_all = []
list_gens = [0,50,100]
list_lsc = ['LLS','RLS','HLS']
list_gen_hydro = [400,500,600] #dry, average, wet hydological scenarios
for gen in list_gens:
    for gen_hy in list_gen_hydro:
        for lsc in list_lsc:
            file_csv_out = 'savnw_sol_' + str(gen) +'_hy_' +str(gen_hy) +'_' + str(lsc)+'_l.csv'
            data_load = pd.read_csv(file_csv_out,header=1, skiprows=0)
            # Extracted data contains branch loading from the metered and non-metered ends. The data is filtered to obtain the branch loading from the metered end.
            data_fil = data_load
            data_fil['From Bus V'] = data_fil['From Bus Ext'].str.extract(r'(\d+\.\d+)') 
            data_fil['To Bus V'] = data_fil['To Bus Ext'].str.extract(r'(\d+\.\d+)')
            data_fil['Scenario']= 'Solar = ' + str(gen) + ' MW, ' +'Hydro = ' +str(gen_hy) +' MW, ' + lsc.upper()
            list_all.append(data_fil)
data_all = pd.concat(list_all).reset_index(drop=True)
data_all = data_all[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID',
       'Non Metered Bus', ' Branch I', 'Branch Flow MW', 'Branch Flow MVAR',
       'Branch MVA', 'MW Loss', 'MVAR Loss', 'Rate', '%I', '%MVA',
       'From Bus V', 'To Bus V']]

##### Line and Transformer Loading

Transformer and branch loading are categorised to following brackets based on the range to which it falls into 
* Loading less than 25%
* Loading 25% to 50%
* Loading 50% to 70%
* Loading 70% to 80%
* Loading 80% to 85%
* Loading 85% to 90%
* Loading 90% to 95%
* Loading 95% to 100%
* Loading 100% and above

Separating transformer and line branch loadings

In [3]:
transformer_loading = data_all[data_all['From Bus V']!=data_all['To Bus V']].reset_index(drop=True)
branch_loading = data_all[data_all['From Bus V'] == data_all['To Bus V']].reset_index(drop=True)

Defining loading brackets for transformer and line branches 

In [4]:
data = transformer_loading[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID','Branch Flow MW', 'Branch Flow MVAR','Branch MVA', 'Rate','%MVA']]
data['Loading'] = ''
data.loc[(data['%MVA'] >100), 'Loading'] = '%MVA>100'
data.loc[(data['%MVA'] <100), 'Loading'] = '95<%MVA<100'
data.loc[(data['%MVA'] <95), 'Loading'] = '90<%MVA<95'
data.loc[(data['%MVA'] <90), 'Loading'] = '85<%MVA<90'
data.loc[(data['%MVA'] <85), 'Loading'] = '80<%MVA<85'
data.loc[(data['%MVA'] <80), 'Loading'] = '70<%MVA<80'
data.loc[(data['%MVA'] <70), 'Loading'] = '50<%MVA<70'
data.loc[(data['%MVA'] <50), 'Loading'] = '25<%MVA<50'
data.loc[(data['%MVA'] <25), 'Loading'] = '%MVA<25'

<ipython-input-4-7f16dd71236d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Loading'] = ''


In [5]:
data_l = branch_loading[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID','Branch Flow MW', 'Branch MVA', 'Rate','%I']]
data_l['Loading'] = ''
data_l.loc[(data_l['%I'] >100), 'Loading'] = '%I>100'
data_l.loc[(data_l['%I'] <100), 'Loading'] = '95<%I<100'
data_l.loc[(data_l['%I'] <95), 'Loading'] = '90<%I<95'
data_l.loc[(data_l['%I'] <90), 'Loading'] = '85<%I<90'
data_l.loc[(data_l['%I'] <85), 'Loading'] = '80<%I<85'
data_l.loc[(data_l['%I'] <80), 'Loading'] = '70<%I<80'
data_l.loc[(data_l['%I'] <70), 'Loading'] = '50<%I<70'
data_l.loc[(data_l['%I'] <50), 'Loading'] = '25<%I<50'
data_l.loc[(data_l['%I'] <25), 'Loading'] = '%I<25'

<ipython-input-5-abcf08d21c83>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_l['Loading'] = ''


##### Histogram Transformer Branch Loading 

A histogram for each scenario is plotted to check the no of transformer branches in the above defined loading bracket 

In [6]:
alt.Chart(data).mark_bar().encode(
    alt.Y('Loading',scale = alt.Scale(domain=['%MVA>100','95<%MVA<100','90<%MVA<95','85<%MVA<90','80<%MVA<85','70<%MVA<80','50<%MVA<70','25<%MVA<50','%MVA<25'])),
    alt.X('count()')).properties(height = 120, width = 250).facet('Scenario',columns=5).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that for most of scenarios the transformer branch loading falls into a bracket of loading between 50 to 70 percentage. 

##### Histogram Line Branch Loading 

A histogram is plotted to check the no of line branches in the above defined loading bracket 

In [7]:
alt.Chart(data_l).mark_bar().encode(
    alt.Y('Loading', scale =alt.Scale(domain=['%I>100','95<%I<100','90<%I<95','85<%I<90','80<%I<85','70<%I<80','50<%I<70','25<%I<50','%I<25'])),
    alt.X('count()')).properties(height = 120, width = 250).facet('Scenario',columns=5).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that for most of scenarios the line branch loading falls into a bracket of loading between 25% to 50%

In the following section the lines and transformer branches with loading greater than 80% is filtered 

##### Checking transformer loading greater than 80% 

In [8]:
transformer_overload  = transformer_loading[transformer_loading['%MVA'] > 80.0].reset_index(drop=True)
transformer_overload['Branch'] = transformer_overload['From Bus'].astype(str) + ' ' + transformer_overload['From Bus Ext'] + ' to ' + \
transformer_overload['To Bus'].astype(str) + ' '+ transformer_overload['To Bus Ext'] + ' ' + transformer_overload['Ckt ID'].astype(str)
list_branch = transformer_overload['Branch'].unique()
print(list_branch)

['202 EAST500     500.00 to 203 EAST230     230.00 1'
 '204 SUB500      500.00 to 205 SUB230      230.00 1'
 '205 SUB230      230.00 to 204 SUB500      500.00 1'
 '3018 CATDOG_G    13.800 to 3008 CATDOG      230.00 1'
 '203 EAST230     230.00 to 202 EAST500     500.00 1']


In [9]:
transformer_overload

,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V,Branch
0,"Solar = 0 MW, Hydro = 400 MW, LLS",202,EAST500 500.00,203,EAST230 230.00,1,203,733,613.40,219.17,651.38,1.62,65.63,800.0,79.44,81.42,500.00,230.00,202 EAST500 500.00 to 203 EAST230 230....
1,"Solar = 0 MW, Hydro = 400 MW, LLS",204,SUB500 500.00,205,SUB230 230.00,1,205,756,638.46,140.29,653.69,1.29,64.31,800.0,81.85,81.71,500.00,230.00,204 SUB500 500.00 to 205 SUB230 230....
2,"Solar = 0 MW, Hydro = 400 MW, LLS",205,SUB230 230.00,204,SUB500 500.00,1,205,1643,-637.17,-75.98,641.68,1.29,64.31,800.0,81.85,80.21,230.00,500.00,205 SUB230 230.00 to 204 SUB500 500....
3,"Solar = 0 MW, Hydro = 400 MW, LLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4801,90.00,80.00,120.42,0.03,11.20,150.0,76.51,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
4,"Solar = 0 MW, Hydro = 400 MW, RLS",202,EAST500 500.00,203,EAST230 230.00,1,203,767,632.20,238.77,675.79,1.77,71.75,800.0,83.06,84.47,500.00,230.00,202 EAST500 500.00 to 203 EAST230 230....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,"Solar = 100 MW, Hydro = 600 MW, HLS",202,EAST500 500.00,203,EAST230 230.00,1,203,934,729.65,320.44,796.91,2.62,106.42,800.0,101.16,99.61,500.00,230.00,202 EAST500 500.00 to 203 EAST230 230....
120,"Solar = 100 MW, Hydro = 600 MW, HLS",203,EAST230 230.00,202,EAST500 500.00,1,203,2031,-727.03,-214.02,757.88,2.62,106.42,800.0,101.16,94.73,230.00,500.00,203 EAST230 230.00 to 202 EAST500 500....
121,"Solar = 100 MW, Hydro = 600 MW, HLS",204,SUB500 500.00,205,SUB230 230.00,1,205,1003,790.15,231.20,823.28,2.26,113.19,800.0,108.58,102.91,500.00,230.00,204 SUB500 500.00 to 205 SUB230 230....
122,"Solar = 100 MW, Hydro = 600 MW, HLS",205,SUB230 230.00,204,SUB500 500.00,1,205,2180,-787.88,-118.01,796.67,2.26,113.19,800.0,108.58,99.58,230.00,500.00,205 SUB230 230.00 to 204 SUB500 500....


##### Transformer branch overload plot

In [10]:
alt.Chart(transformer_overload).mark_line().encode(
    alt.X('%MVA', axis=alt.Axis(format='.3f'),scale=alt.Scale(zero=False)),
    alt.Y('Scenario', sort='-x'),
    alt.Color('Branch', legend=None)).properties(height = 300, width =400).facet('Branch',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

##### Checking branch loading greater than 80% 

In [11]:
branch_overload = branch_loading[branch_loading['%I'] > 80.0].reset_index(drop=True)
branch_overload['Branch'] = branch_overload['From Bus'].astype(str) + ' ' + branch_overload['From Bus Ext'] + ' to ' + \
branch_overload['To Bus'].astype(str) + ' '+ branch_overload['To Bus Ext'] + ' ' + branch_overload['Ckt ID'].astype(str)
list_branch = branch_overload['Branch'].unique()
print(list_branch)

['153 MID230      230.00 to 154 DOWNTN      230.00 1'
 '154 DOWNTN      230.00 to 153 MID230      230.00 1'
 '3001 MINE        230.00 to 3003 S. MINE     230.00 1'
 '3003 S. MINE     230.00 to 3001 MINE        230.00 1'
 '3005 WEST        230.00 to 3007 RURAL       230.00 1'
 '3007 RURAL       230.00 to 3005 WEST        230.00 1'
 '153 MID230      230.00 to 154 DOWNTN      230.00 2'
 '154 DOWNTN      230.00 to 153 MID230      230.00 2'
 '154 DOWNTN      230.00 to 203 EAST230     230.00 1'
 '203 EAST230     230.00 to 154 DOWNTN      230.00 1']


In [12]:
branch_overload

,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V,Branch
0,"Solar = 0 MW, Hydro = 400 MW, LLS",153,MID230 230.00,154,DOWNTN 230.00,1,154,673,261.75,73.65,271.91,3.64,32.74,300.0,89.44,90.64,230.00,230.00,153 MID230 230.00 to 154 DOWNTN 230....
1,"Solar = 0 MW, Hydro = 400 MW, LLS",154,DOWNTN 230.00,153,MID230 230.00,1,154,679,-258.11,-50.77,263.06,3.64,32.74,300.0,90.21,87.69,230.00,230.00,154 DOWNTN 230.00 to 153 MID230 230....
2,"Solar = 0 MW, Hydro = 400 MW, RLS",153,MID230 230.00,154,DOWNTN 230.00,1,154,703,269.44,80.74,281.28,3.97,35.70,300.0,93.39,93.76,230.00,230.00,153 MID230 230.00 to 154 DOWNTN 230....
3,"Solar = 0 MW, Hydro = 400 MW, RLS",154,DOWNTN 230.00,153,MID230 230.00,1,154,709,-265.47,-54.69,271.05,3.97,35.70,300.0,94.19,90.35,230.00,230.00,154 DOWNTN 230.00 to 153 MID230 230....
4,"Solar = 0 MW, Hydro = 400 MW, RLS",3001,MINE 230.00,3003,S. MINE 230.00,1,3003,511,200.39,63.78,210.29,0.00,3.32,250.0,81.45,84.12,230.00,230.00,3001 MINE 230.00 to 3003 S. MINE 23...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,"Solar = 100 MW, Hydro = 600 MW, HLS",203,EAST230 230.00,154,DOWNTN 230.00,1,203,443,156.34,53.65,165.29,1.27,12.68,200.0,88.25,82.65,230.00,230.00,203 EAST230 230.00 to 154 DOWNTN 230....
190,"Solar = 100 MW, Hydro = 600 MW, HLS",3001,MINE 230.00,3003,S. MINE 230.00,1,3003,622,219.62,127.59,253.99,0.00,4.93,250.0,99.27,101.60,230.00,230.00,3001 MINE 230.00 to 3003 S. MINE 23...
191,"Solar = 100 MW, Hydro = 600 MW, HLS",3003,S. MINE 230.00,3001,MINE 230.00,1,3003,622,-219.62,-122.66,251.55,0.00,4.93,250.0,99.27,100.62,230.00,230.00,3003 S. MINE 230.00 to 3001 MINE 23...
192,"Solar = 100 MW, Hydro = 600 MW, HLS",3005,WEST 230.00,3007,RURAL 230.00,1,3007,742,265.36,111.98,288.02,2.65,22.06,300.0,98.62,96.01,230.00,230.00,3005 WEST 230.00 to 3007 RURAL 23...


##### Transmission line branch overload plot

In [13]:
alt.Chart(branch_overload).mark_line().encode(
    alt.X('%I', scale=alt.Scale(zero=False)),
    alt.Y('Scenario', sort='-x'),
    alt.Color('Branch', legend=None)).properties(height = 300, width =400).facet('Branch',columns=3).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

##### Bus Voltage

In [14]:
list_volt = []
list_gens = [0,50,100]
list_lsc = ['LLS','RLS','HLS']
list_gen_hydro = [400,500,600] #dry, average, wet hydological scenarios
for gen in list_gens:
    for gen_hy in list_gen_hydro:
        for lsc in list_lsc:
            file_sav_in = 'savnw_sol_' + str(gen) +'_hy_' +str(gen_hy) +'_' + str(lsc)+'_volt.csv'
            data_volt = pd.read_csv(file_sav_in,header=1, skiprows=0)
            data_volt['Scenario']= 'Solar = ' + str(gen) + ' MW, ' +'Hydro = ' +str(gen_hy) +' MW, ' + lsc.upper()
            list_volt.append(data_volt)
data_all_volt = pd.concat(list_volt).reset_index(drop=True)
data_all_volt = data_all_volt[['Scenario','Bus Number', 'Bus Name', 'Area', 'Base Voltage', 'Bus Voltage(PU)','Bus Voltage(kV)']]

Similar to branch loading categorization into brackets of loadings, voltages are also categorised into brackets

In [15]:
data_v = data_all_volt
data_v['Voltage Range'] = ''
data_v.loc[(data_v['Bus Voltage(PU)'] >1.1), 'Voltage Range'] = 'Vbus>1.1'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.1), 'Voltage Range'] = '1.05<Vbus<1.1'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.05), 'Voltage Range'] = '1.0<Vbus<1.05'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.0), 'Voltage Range'] = '0.95<Vbus<1.0'
data_v.loc[(data_v['Bus Voltage(PU)'] <0.95), 'Voltage Range'] = '0.95<Vbus<0.9'
data_v.loc[(data_v['Bus Voltage(PU)'] <0.9), 'Voltage Range'] = 'Vbus<0.9'

##### Histogram Bus Voltages

A histogram is plotted to check the no of Buses in each of the above defined brackets 

In [16]:
alt.Chart(data_v).mark_bar().encode(
    alt.Y('Voltage Range', scale =alt.Scale(domain=['Vbus>1.1','1.05<Vbus<1.1','1.0<Vbus<1.05','0.95<Vbus<1.0','0.95<Vbus<0.9','Vbus<0.9'])),
    alt.X('count()')).properties(height = 120, width = 250).facet('Scenario',columns=5).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that most of the buses have voltages in the bracket of 1.0 PU and 1.05 PU

In the following section, the buses out of upper and lower normal operating range is filtered and reported 

##### Voltage Upper Limits 

In [17]:
list_volt = list(data_all_volt[data_all_volt['Bus Voltage(PU)']>1.05].reset_index(drop=True)['Bus Number'].unique())
list_volt

[206, 211, 212]

##### Voltage Upper Limits 

In [18]:
data_all_volt[data_all_volt['Bus Voltage(PU)']>1.05].reset_index(drop=True)['Bus Number'].unique()

array([206, 211, 212], dtype=int64)

In [19]:
volt_high = data_all_volt[data_all_volt['Bus Voltage(PU)']>1.05].reset_index(drop=True)
volt_high

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV),Voltage Range
0,"Solar = 0 MW, Hydro = 500 MW, LLS",206,URBGEN,2,18.0,1.053,18.948,1.05<Vbus<1.1
1,"Solar = 0 MW, Hydro = 500 MW, HLS",211,HYDRO_G,2,20.0,1.061,21.228,1.05<Vbus<1.1
2,"Solar = 0 MW, Hydro = 500 MW, HLS",212,HYDRO_N,2,20.0,1.061,21.228,1.05<Vbus<1.1
3,"Solar = 0 MW, Hydro = 600 MW, RLS",211,HYDRO_G,2,20.0,1.062,21.237,1.05<Vbus<1.1
4,"Solar = 0 MW, Hydro = 600 MW, RLS",212,HYDRO_N,2,20.0,1.062,21.237,1.05<Vbus<1.1
5,"Solar = 0 MW, Hydro = 600 MW, HLS",211,HYDRO_G,2,20.0,1.080,21.609,1.05<Vbus<1.1
6,"Solar = 0 MW, Hydro = 600 MW, HLS",212,HYDRO_N,2,20.0,1.080,21.609,1.05<Vbus<1.1
7,"Solar = 50 MW, Hydro = 500 MW, HLS",211,HYDRO_G,2,20.0,1.057,21.131,1.05<Vbus<1.1
8,"Solar = 50 MW, Hydro = 500 MW, HLS",212,HYDRO_N,2,20.0,1.057,21.131,1.05<Vbus<1.1
9,"Solar = 50 MW, Hydro = 600 MW, RLS",211,HYDRO_G,2,20.0,1.057,21.138,1.05<Vbus<1.1


##### Upper voltage limit violation plot

In [20]:
alt.Chart(volt_high).mark_line().encode(
    alt.X('Bus Voltage(PU)', axis=alt.Axis(format='.3f'), scale=alt.Scale(nice=False, zero=False)),
    alt.Y('Scenario', sort='-x'),
    alt.Color('Bus Number:N', legend=None)).properties(height = 150, width = 250).facet('Bus Number:N',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

In [21]:
data_all_volt[data_all_volt['Bus Voltage(PU)']<0.95].reset_index(drop=True)['Bus Number'].unique()

array([ 154,  205,  103,  203, 3007, 3008,  204], dtype=int64)

##### Voltage Lower Limits

In [22]:
volt_low = data_all_volt[data_all_volt['Bus Voltage(PU)']<0.95].reset_index(drop=True)
volt_low

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV),Voltage Range
0,"Solar = 0 MW, Hydro = 400 MW, HLS",154,DOWNTN,1,230.0,0.931,214.213,0.95<Vbus<0.9
1,"Solar = 0 MW, Hydro = 400 MW, HLS",205,SUB230,2,230.0,0.941,216.472,0.95<Vbus<0.9
2,"Solar = 0 MW, Hydro = 500 MW, RLS",154,DOWNTN,1,230.0,0.945,217.455,0.95<Vbus<0.9
3,"Solar = 0 MW, Hydro = 500 MW, HLS",103,SOLAR_PV,1,13.8,0.946,13.057,0.95<Vbus<0.9
4,"Solar = 0 MW, Hydro = 500 MW, HLS",154,DOWNTN,1,230.0,0.916,210.790,0.95<Vbus<0.9
5,"Solar = 0 MW, Hydro = 500 MW, HLS",203,EAST230,2,230.0,0.945,217.303,0.95<Vbus<0.9
6,"Solar = 0 MW, Hydro = 500 MW, HLS",205,SUB230,2,230.0,0.926,213.079,0.95<Vbus<0.9
7,"Solar = 0 MW, Hydro = 500 MW, HLS",3007,RURAL,5,230.0,0.945,217.277,0.95<Vbus<0.9
8,"Solar = 0 MW, Hydro = 500 MW, HLS",3008,CATDOG,5,230.0,0.938,215.770,0.95<Vbus<0.9
9,"Solar = 0 MW, Hydro = 600 MW, RLS",154,DOWNTN,1,230.0,0.926,212.988,0.95<Vbus<0.9


##### Lower voltage limit violation plot

In [23]:
alt.Chart(volt_low).mark_line().encode(
    alt.X('Bus Voltage(PU)', axis=alt.Axis(format='.3f'), scale=alt.Scale(nice=False, zero=False)),
    alt.Y('Scenario', sort='x'),
    alt.Color('Bus Number:N', legend=None)).properties(height = 150, width = 250).facet('Bus Number:N',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

#### Conclusion 

Year 3, Toplogy 3 limit violation check for overloading and voltage was carried out for the three load scenarios. It was seen that
1. overloading of greater than 80% was reported for following branches and transformers for various solar output and load scenarios 
    * Transformer 204 SUB500      500.00 to 205 SUB230      230.00 1
    * Transformer 205 SUB230      230.00 to 204 SUB500      500.00 1
    * Transformer 3018 CATDOG_G    13.800 to 3008 CATDOG      230.00 1
    * Transformer 203 EAST230     230.00 to 202 EAST500     500.00 1
    * Branch 153 MID230      230.00 to 154 DOWNTN      230.00 1
     * Branch 154 DOWNTN      230.00 to 153 MID230      230.00 1
     * Branch 3001 MINE        230.00 to 3003 S. MINE     230.00 1
     * Branch 3003 S. MINE     230.00 to 3001 MINE        230.00 1
     * Branch 3005 WEST        230.00 to 3007 RURAL       230.00 1
     * Branch 3007 RURAL       230.00 to 3005 WEST        230.00 1
     * Branch 153 MID230      230.00 to 154 DOWNTN      230.00 2
     * Branch 154 DOWNTN      230.00 to 153 MID230      230.00 2
     * Branch 154 DOWNTN      230.00 to 203 EAST230     230.00 1
     * Branch 203 EAST230     230.00 to 154 DOWNTN      230.00 1
2. The buses violating the upper voltage limit for various scenarios are
    * Buses 211, 212
3. The buses violating the lower voltage limit for various scenarios are
    * Buses 154,  205,  103,  203, 3007, 3008,  204